# Debening

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append("../")

In [2]:
import re
import json
from gensim.models import Word2Vec

In [3]:
from Blockade import get_resource_path, get_data_path

In [20]:
poem_path = get_resource_path('debening', "poem300.txt")
with open(poem_path, "r", encoding="UTF-8") as fin:
    poems = fin.readlines()
poems = [x.strip() for x in poems if not re.match(r"^\d", x)]
poems = ["，".join(poems[i:i+4])+"。" for i in range(0, len(poems), 4)]

In [21]:
poems[0]

'孤鴻海上來，池潢不敢顧，側見雙翠鳥，巢在三珠樹。'

In [93]:
classic_path_templ = str(get_resource_path('debening', 'volume{vol_i}.txt'))
classics = []
def keepCJK(x):
    return "".join(re.findall(r"[\u4e00-\u9fff，。]", x))
for vi in range(1, 13):
    classic_path = classic_path_templ.format(vol_i=vi)
    print(classic_path)
    fin = open(classic_path, "r", encoding="UTF-8")
    data = json.load(fin)
    text = [keepCJK(x.strip()) for x in data["content"] if not re.match("《", x)]
    text = classics.extend(text)
    fin.close()

/Users/seantyh/LangOn/Blockade/resources/debening/volume1.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume2.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume3.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume4.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume5.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume6.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume7.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume8.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume9.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume10.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume11.txt
/Users/seantyh/LangOn/Blockade/resources/debening/volume12.txt


In [94]:
textbase = poems + classics

In [95]:
wv_model = Word2Vec(textbase, size=100, window=5, min_count=1)

In [96]:
stoi = {s: i for i, s in enumerate(wv_model.wv.index2word)}
itos = {v: k for k, v in stoi.items()}

In [97]:
syn1neg = wv_model.trainables.syn1neg
syn1neg.shape

(4050, 100)

In [98]:
wv_model.wv.vectors.shape

(4050, 100)

In [99]:
import pickle
syn1neg_path = get_data_path('debening', 'syn1neg.pkl')
with open(syn1neg_path, "wb") as fout:
    pickle.dump((syn1neg, itos, stoi), fout)

In [100]:
def vec2char(vec):
    return itos[np.argmax(np.dot(syn1neg, vec))]
    

In [144]:
"".join([vec2char(np.random.beta(1,5,100)) for _ in range(4)])

'歉歉歉歉'